In [1]:
from vpython import*

<IPython.core.display.Javascript object>

In [2]:
scene1 = canvas()

<IPython.core.display.Javascript object>

In [3]:
class Net:
    def __init__(self, driver, name, on=0, check=0):
        self.value = None
        self.driver = driver
        self.name = name
        self.check = check
        self.receivers = []
        self.on = on 

    def connect(self, inputs):
        if not isinstance(inputs, list):
            inputs = [inputs]
            
        for input in inputs:
            self.receivers.append(input)

    def set(self, value):
        if self.value == value:
            return 
        
        self.value = value
        
        if self.on:
            self.driver.evaluate()
            
        if self.check:
            print("Connector {0}-{1} value = {2}".format(self.driver.name, self.name, self.value))
        for c in self.receivers:
            c.set(value)
        
class Logic:
    def __init__(self, name):
        self.name = name

    def evaluate(self):
        return

class Gate(Logic):        
    def __init__(self, name):
        Logic.__init__(self, name)
        self.A = Net(self, 'Input1', on=1)
        self.B = Net(self, 'Input2', on=1)
        self.C = Net(self, 'Output')

class Nand(Gate):       
    def __init__(self, name):
        Gate.__init__(self, name)
        
    def evaluate(self):
        result = not(self.A.value and self.B.value)
        self.C.set(result)
        
#     def make_gate(self):
#         gate = curve(color=color.white, radius=.5)
#         gate.append(vector(0,0,0))
#         gate.append(pos=vector(.5,0,0))
#         sphere(pos=vector(1,0,0), radius=.1)

class Nor(Gate):         
    def __init__(self, name):
        Gate.__init__(self, name)

    def evaluate(self):
        result = not(self.A.value and self.B.value)
        self.C.set(result)
        
def setInputs(object, a,b):
    object.A.set(a)
    object.B.set(b)

In [4]:
nand = Nand('nand1')
nor = Nor('nor1')



# for val in truth_values:
#     a,b = val
#     setInputs(nand, a, b)
#     print(nand.C.value)

In [5]:
scene2 = canvas()

<IPython.core.display.Javascript object>

In [6]:
truth_values = [(0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1), (0,0), (0,1),
                (1,0), (1,1)]
t = []

for i in range(12):
    t.append(i)

gate_graph = graph(title="Nand Gate", xtitle='time', ytitle='value', fast=False, width=800)
gate1 = gcurve(color=color.blue, width=4, markers=True, marker_color=color.black, label='nand1')


for i in range(40):
    rate(1)
    a,b = truth_values[i]
    setInputs(nand, a, b)
    gate1.plot(t[i], nand.C.value)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

IndexError: list index out of range